# Nuclia

>[Nuclia](https://nuclia.com) 可自动索引来自任何内部和外部来源的非结构化数据，提供优化的搜索结果和生成式答案。它可以处理视频和音频转录、图像内容提取以及文档解析。

> `Nuclia Understanding API` 支持处理包括文本、网页、文档以及音视频内容在内的非结构化数据。它会提取所有文本（根据需要使用语音转文本或 OCR），还会提取元数据、嵌入文件（如下载的 PDF 中的图片）以及网页链接。如果启用了机器学习，它还能识别实体、提供内容摘要并为所有句子生成嵌入。

## 设置

要使用 `Nuclia Understanding API`，您需要有一个 Nuclia 账户。您可以免费在 [https://nuclia.cloud](https://nuclia.cloud) 创建一个账户，然后 [创建 NUA 密钥](https://docs.nuclia.dev/docs/docs/using/understanding/intro)。

In [ ]:
%pip install --upgrade --quiet  protobuf
%pip install --upgrade --quiet  nucliadb-protos

In [3]:
import os

os.environ["NUCLIA_ZONE"] = "<YOUR_ZONE>"  # e.g. europe-1
os.environ["NUCLIA_NUA_KEY"] = "<YOUR_API_KEY>"

## 示例

要使用 Nuclia 文档加载器，您需要实例化一个 `NucliaUnderstandingAPI` 工具：

In [ ]:
from langchain_community.tools.nuclia import NucliaUnderstandingAPI

nua = NucliaUnderstandingAPI(enable_ml=False)

In [ ]:
from langchain_community.document_loaders.nuclia import NucliaLoader

loader = NucliaLoader("./interview.mp4", nua)

您现在可以在循环中调用 `load` 方法，直到获取到文档。

In [ ]:
import time

pending = True
while pending:
    time.sleep(15)
    docs = loader.load()
    if len(docs) > 0:
        print(docs[0].page_content)
        print(docs[0].metadata)
        pending = False
    else:
        print("waiting...")

## 获取的信息

Nuclia 返回以下信息：

- 文件元数据
- 提取的文本
- 嵌套文本（例如嵌入图像中的文本）
- 段落和句子拆分（由其首尾字符位置，以及视频或音频文件的开始和结束时间定义）
- 链接
- 缩略图
- 嵌入式文件

注意：

  生成的文件（缩略图、提取的嵌入式文件等）以 token 的形式提供。您可以使用 [`/processing/download` 端点](https://docs.nuclia.dev/docs/api#operation/Download_binary_file_processing_download_get) 下载它们。

  同样，在任何层级，如果某个属性超过一定大小，它将被放入一个可下载的文件中，并被文件指针替换。该文件指针将由 `{"file": {"uri": "JWT_TOKEN"}}` 组成。规则是，如果消息大小超过 1000000 个字符，最大的部分将被移至可下载的文件中。首先，压缩过程将以 vectors 为目标。如果这还不够，它将以大型字段元数据为目标，最后将以提取的文本为目标。